The idea of computing the value of the down_and_out european option is similar to computing the value of the asian option. 

Assume that the maturity date is given in $T$. We then divide the interval $(0,T$], into Steps many equal parts. Then we estimate the value of the underlying asset for each end point of these intervals. Multiplier variable is equal to zero, if at one of these point, the value of the underlying asset falls below the Barrier (B). If the value of the underlying asset never falls below $B$, the Multiplier is equal to 1.

The final value of the the down and out option (in each simulation) is given by
\begin{align*}
e^{-rT}  .Multiplier .\dot(X(T)-K)^+
\end{align*}
Then, to estimate the value of this option we take the expected value of these prices over $40000$ simulaitons.

In [2]:


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sp

np.random.seed(1)

In [14]:
class EuropeanOption:
    def __init__(self, Barrier, Maturity = 1, InterestRate = 0.05,
              Sigma = 0.3, InitialValue = 50.0, StrikePrice = 55):
        self.Maturity = Maturity
        self.InterestRate = InterestRate
        self.Sigma = Sigma
        self.InitialValue = InitialValue
        self.StrikePrice = StrikePrice
        self.Sigma2 = self.Sigma * Sigma / 2
        self.Barrier = Barrier
        
    def CI(self, data,confidence = 0.975):
        arr = np.array(data)
        n = len(arr)
        mean = np.mean(arr)
        sd = np.std(arr,ddof = 1)
        z_value = sp.norm.ppf(.975)
        hw = z_value * sd / np.sqrt(n)
        return mean, [mean-hw,mean + hw]
    
    def PriceSimulation(self, Steps = 64):
        Interval = self.Maturity / Steps
        X = self.InitialValue
        Multiplier = 1
        for i in range(Steps):
            Z = np.random.standard_normal(1)
            X = X * np.exp((self.InterestRate - self.Sigma2) * Interval +
                       self.Sigma * np.sqrt(Interval) * Z)
            m = 1

            Multiplier = min (Multiplier, X> self.Barrier)
        Value = Multiplier * np.exp(-self.InterestRate * self.Maturity) * max (X,0)
        #print(Value)


        return Value
    
    def PriceStats(self,Replication, Steps = 64):
        ValueList = []
        for i in range(Replication):
            ValueList.append(self.PriceSimulation(64))
        #print(ValueList)
        print(self.CI(ValueList))
            

In [20]:
Barriers = [35,40,45]
for B in Barriers:
    option = EuropeanOption(B)
    print('For B = {} the estimated value and 95 percent CI is: '.format(B),)
    option.PriceStats(Replication = 40000,Steps = 64)


For B = 35 the estimated value and 95 percent CI is: 
(43.09105316076773, [42.845082344794626, 43.33702397674083])
For B = 40 the estimated value and 95 percent CI is: 
(34.45807343122589, [34.159513174618354, 34.75663368783343])
For B = 45 the estimated value and 95 percent CI is: 
(20.956585747776874, [20.655529342980664, 21.257642152573084])


Note that as the value of the Barrier increases, the value of the option decreases. This is because as the Barrier increases, it becomes more likely for the asset price to fall below the Barrier value.